In [2]:
!pip install -q datasets
!huggingface-cli login

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  

In [6]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('lavita/ChatDoctor-HealthCareMagic-100k')

# Mélanger le dataset et en sélectionner un échantillon
dataset = dataset['train'].shuffle(seed=42).select(range(1000))

# Définir une fonction pour transformer les données
def transform_chatdoctor(example):
    # Vérifier si les champs 'input' et 'output' existent et ne sont pas vides
    question = example.get('input', '').strip()
    answer = example.get('output', '').strip()

    if not question or not answer:
        # Retourner None pour ignorer les exemples mal formatés
        return None

    # Appliquer le nouveau modèle de formatage
    return {'text': f'<s>[INST] {question} [/INST] {answer} </s>'}

# Appliquer la transformation, en filtrant les exemples non valides
transformed_dataset = dataset.map(transform_chatdoctor, remove_columns=dataset.column_names, batched=False)

# Vérifier un échantillon
print(transformed_dataset[0])


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'text': '<s>[INST] I have been having alot of catching ,pain and discomfort under my right rib.  If I twist to either side especially my right it feels like my rib actually catches on something and at times I have to stop try to catch my breath and wait for it to subside.  There are times if I am laughing too hard that it will do the same thing but normally its more so if I have twisted or moved  a certain way [/INST] Hi thanks for asking question. Here you are complaining pain in particular position esp. While turning to a side. So strong possibility is about moderate degree muscular strain. It might have occurred by heavyweight lift or during some activities. Simple analgesic taken. Take rest. Sleep in supine position. Second here Costco Chat Doctor.  Ribs are tender to touch.x-ray also useful. If cough, cold, sore throat present then respiratory infections also has to be ruled out. Treat it symptomatically. If still seems serious then x-ray done for chest. CBC will also be done. If

In [8]:
transformed_dataset.push_to_hub("ChatDoctor-HealthCareMagic-llama2-100k")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jassercherif/ChatDoctor-HealthCareMagic-llama2-100k/commit/92b73269790369e43993c55ddd17e6eee19c23dd', commit_message='Upload dataset', commit_description='', oid='92b73269790369e43993c55ddd17e6eee19c23dd', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/jassercherif/ChatDoctor-HealthCareMagic-llama2-100k', endpoint='https://huggingface.co', repo_type='dataset', repo_id='jassercherif/ChatDoctor-HealthCareMagic-llama2-100k'), pr_revision=None, pr_num=None)